# A 2D Infinite Fuel Pin Geometry of the SNRE 

## Material Definitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import openmc

openmc.Materials.cross_sections = "/home/jimmy/Documents/SNRE_UROP/SNRE/ICL_UROP/dataLibrary/cross_sections.xml"

# Materials definitions
fuel = openmc.Material(name='fuel', temperature=300)
fuel.set_density('g/cm3', 3.64)
fuel.add_nuclide('U235', 0.558, 'wo')
fuel.add_nuclide('U238', 0.042, 'wo')
fuel.add_nuclide('Zr0', 1.81, 'wo')
fuel.add_nuclide('C0', 1.23, 'wo')
fuel.add_s_alpha_beta('c_Graphite')

coolant = openmc.Material(name='coolant', temperature=300)
coolant.set_density('atom/b-cm', 1.61317)
coolant.add_nuclide('H1', 1.0, 'wo')
coolant.add_s_alpha_beta('c_para_H')

cladding = openmc.Material(name='cladding', temperature=300)
cladding.set_density('g/cm3', 6.73)
cladding.add_nuclide('Zr0', 0.88375, 'wo')
cladding.add_nuclide('C0', 0.11625, 'wo')
cladding.add_s_alpha_beta('c_Graphite')

# Materials xml
materials = openmc.Materials([fuel, coolant, cladding])
materials.export_to_xml('./xmlFiles/')

## Fuel Element Geometry Definitions

In [ ]:
# Geometry definitions
channelRadius = 0.11827
cladOuterRadius = 0.12827
pinPitch = 0.40894
subassemblyDuctOuter = 1.905
subassemblyDuctThickness = 0.005
subassemblyDuctInner = subassemblyDuctOuter - 2*subassemblyDuctThickness

# Boudaries and outer universe
fuelOut = openmc.Cell(cell_id=200, fill=fuel)

# Inner Hexagon
assemblyHex = openmc.hexagonal_prism(edge_length=subassemblyDuctInner / np.sqrt(3.),
                                        orientation='x')

# Outer Hexagon
cladOuterHex = openmc.hexagonal_prism(edge_length=subassemblyDuctOuter / np.sqrt(3.), 
                                         orientation='x',
                                         boundary_type='reflective')

assemblyCell = openmc.Cell()
cladOuterCell = openmc.Cell()
voidCell = openmc.Cell()

assemblyCell.region = assemblyHex
cladOuterCell.region = ~assemblyHex & cladOuterHex 
voidCell.region = ~cladOuterHex
cladOuterCell.fill = cladding

# Create universes
fuelOutUni = openmc.Universe(cells=(fuelOut,))
cladOuterUni = openmc.Universe(cells=[cladOuterCell])
rootUni = openmc.Universe()

coolantChannelOuter = openmc.ZCylinder(surface_id=400, r=channelRadius)
cladChannelOuter = openmc.ZCylinder(surface_id=402, r=cladOuterRadius)

coolantChannelCell  = openmc.Cell(cell_id=400, fill=coolant, region=-coolantChannelOuter)
cladChannelCell  = openmc.Cell(cell_id=401, fill=cladding, region=+coolantChannelOuter & -cladChannelOuter)
fuelChannelCell = openmc.Cell(cell_id=402, fill=fuel, region=+cladChannelOuter)

channelUni = openmc.Universe(cells=[fuelChannelCell, cladChannelCell, coolantChannelCell])

# Creating the hexagonal lattice
lattice=openmc.HexLattice(name='assembly')
lattice.center = (0., 0.)
lattice.pitch = (pinPitch,)
lattice.outer = fuelOutUni
ring2=[channelUni]*12
ring1=[channelUni]*6
ring0=[channelUni]
lattice.universes = [ring2, ring1, ring0]
lattice.orientation='x'
assemblyCell.fill = lattice

rootUni.add_cells([assemblyCell, cladOuterCell, voidCell])
geom=openmc.Geometry(rootUni)
geom.export_to_xml('./xmlFiles/')

# Plotting the fuel pin geometry
rootUni.plot(origin=(0,0,0),width=(2.2,2.2),color_by='material',colors={fuel: 'yellow', coolant: 'skyblue', cladding: 'black',},pixels=[1000,1000])
plt.savefig('./pics/Geometry/fuelPinCrossSection.png', dpi=300)

## Support Element Geometry Definitions

In [ ]:
# # Geometry definitions
# channelRadius = 0.11827
# cladOuterRadius = 0.12827
# pinPitch = 0.40894
# subassemblyDuctOuter = 1.905
# subassemblyDuctThickness = 0.005
# subassemblyDuctInner = subassemblyDuctOuter - 2*subassemblyDuctThickness

# # Boudaries and outer universe
# fuelOut = openmc.Cell(cell_id=200, fill=fuel)

# # Inner Hexagon
# assemblyHex = openmc.hexagonal_prism(edge_length=subassemblyDuctInner / np.sqrt(3.),
#                                         orientation='x')

# # Outer Hexagon
# cladOuterHex = openmc.hexagonal_prism(edge_length=subassemblyDuctOuter / np.sqrt(3.), 
#                                          orientation='x',
#                                          boundary_type='reflective')

# assemblyCell = openmc.Cell()
# cladOuterCell = openmc.Cell()
# voidCell = openmc.Cell()

# assemblyCell.region = assemblyHex
# cladOuterCell.region = ~assemblyHex & cladOuterHex 
# voidCell.region = ~cladOuterHex
# cladOuterCell.fill = cladding

# # Create universes
# fuelOutUni = openmc.Universe(cells=(fuelOut,))
# cladOuterUni = openmc.Universe(cells=[cladOuterCell])
# rootUni = openmc.Universe()

# coolantChannelOuter = openmc.ZCylinder(surface_id=400, r=channelRadius)
# cladChannelOuter = openmc.ZCylinder(surface_id=402, r=cladOuterRadius)

# coolantChannelCell  = openmc.Cell(cell_id=400, fill=coolant, region=-coolantChannelOuter)
# cladChannelCell  = openmc.Cell(cell_id=401, fill=cladding, region=+coolantChannelOuter & -cladChannelOuter)
# fuelChannelCell = openmc.Cell(cell_id=402, fill=fuel, region=+cladChannelOuter)

# channelUni = openmc.Universe(cells=[fuelChannelCell, cladChannelCell, coolantChannelCell])

# # Creating the hexagonal lattice
# lattice=openmc.HexLattice(name='assembly')
# lattice.center = (0., 0.)
# lattice.pitch = (pinPitch,)
# lattice.outer = fuelOutUni
# ring2=[channelUni]*12
# ring1=[channelUni]*6
# ring0=[channelUni]
# lattice.universes = [ring2, ring1, ring0]
# lattice.orientation='x'
# assemblyCell.fill = lattice

# rootUni.add_cells([assemblyCell, cladOuterCell, voidCell])
# geom=openmc.Geometry(rootUni)
# geom.export_to_xml('./xmlFiles/')

# # Plotting the fuel pin geometry
# rootUni.plot(origin=(0,0,0),width=(2.2,2.2),color_by='material',colors={fuel: 'yellow', coolant: 'skyblue', cladding: 'black',},pixels=[500,500])

## Setting Simulation Parameters

In [ ]:
# OpenMC simulation parameters
settings = openmc.Settings()
settings.run_mode = 'eigenvalue'

settings.particles = 5000
settings.generations_per_batch = 20
settings.batches = 100
settings.inactive = 10
settings.export_to_xml('./xmlFiles/')

source = openmc.IndependentSource()
source.space = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))

mesh = openmc.RegularMesh()
mesh.dimension = [500, 500]
mesh.lower_left = [-1.1, -1.1]
mesh.upper_right = [1.1, 1.1]

tallies = openmc.Tallies()

fuelAllTally = openmc.Tally(tally_id=1)
fuelAllTally.filters = [openmc.MeshFilter(mesh)]
fuelAllTally.scores = ['flux', 'nu-fission', 'absorption', 'elastic', 'total']
tallies.append(fuelAllTally)

fuelThermalTally = openmc.Tally(tally_id=2)
fuelThermalTally.filters = [openmc.MeshFilter(mesh), openmc.EnergyFilter([0,0.1])]
fuelThermalTally.scores = ['flux', 'nu-fission', 'absorption', 'elastic', 'total']
tallies.append(fuelThermalTally)

fuelFastTally = openmc.Tally(tally_id=3)
fuelFastTally.filters = [openmc.MeshFilter(mesh), openmc.EnergyFilter([0.1,1e6])]
fuelFastTally.scores = ['flux', 'nu-fission', 'absorption', 'elastic', 'total']
tallies.append(fuelFastTally)

tallies.export_to_xml('./xmlFiles/')

openmc.run(path_input='./xmlFiles/')

In [ ]:
with openmc.StatePoint('statepoint.{}.h5'.format(settings.batches)) as sp:
    outputAllTally = sp.get_tally(id=1)
    dfAll = outputAllTally.get_pandas_dataframe()
    dfAllFlux = dfAll[dfAll['score'] == 'flux']
    dfAllFission = dfAll[dfAll['score'] == 'nu-fission']
    dfAllAbsorption = dfAll[dfAll['score'] == 'absorption']
    #dfAllElastic = dfAll[dfAll['score'] == '(n,elastic)']
    dfAllReactionRate = dfAll[dfAll['score'] == 'total']
    
# print(outputAllTally)
# print(dfAll)
# print(dfAllFlux)
# print(dfAllFlux['mean'])
# print(dfAllFission)
# print(dfAllFission['mean'])
# print(dfAllAbsorption)
# print(dfAllAbsorption['mean'])
# # print(dfAllElastic)
# # print(dfAllElastic['mean'])
# print(dfAllReactionRate)
# print(dfAllReactionRate['mean'])


meanAllFlux = dfAllFlux['mean'].values.reshape((500,500))
plt.imshow(meanAllFlux, interpolation='spline16', cmap='turbo')
plt.title('All Flux')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.savefig('./pics/All/AllFlux.png', dpi=300)
#plt.clim(0.0008,0.0012)
plt.show()

meanAllFluxAx = np.column_stack((np.arange(500), meanAllFlux[250]))
np.savetxt('./dataFiles/meanAllFluxAx.csv', meanAllFluxAx, delimiter=',', fmt='%s')
meanAllFluxAxData, meanAllFluxAxPos = meanAllFluxAx[:,0], meanAllFluxAx[:,1]
plt.plot(meanAllFluxAxData,meanAllFluxAxPos)
plt.title('All Flux')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Flux')
plt.savefig('./pics/All/AllFluxAx.png', dpi=300)
plt.show()

meanAllFission = dfAllFission['mean'].values.reshape((500,500))
plt.imshow(meanAllFission, interpolation='spline16', cmap='turbo')
plt.title('All nu-fission')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/All/AllFission.png', dpi=300)
plt.show()

meanAllFissionAx = np.column_stack((np.arange(500), meanAllFission[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanAllFissionAx, delimiter=',', fmt='%s')
meanAllFissionAxData, meanAllFissionAxPos = meanAllFissionAx[:,0], meanAllFissionAx[:,1]
plt.plot(meanAllFissionAxData,meanAllFissionAxPos)
plt.title('All nu-fission')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('nu-fission')
plt.savefig('./pics/All/AllFissionAx.png', dpi=300)
plt.show()

meanAllAbsorption = dfAllAbsorption['mean'].values.reshape((500,500))
plt.imshow(meanAllAbsorption, interpolation='spline16', cmap='turbo')
plt.title('All Absorption')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/All/AllAbsorption.png', dpi=300)
plt.show()

meanAllAbsorptionAx = np.column_stack((np.arange(500), meanAllAbsorption[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanAllAbsorptionAx, delimiter=',', fmt='%s')
meanAllAbsorptionAxData, meanAllAbsorptionAxPos = meanAllAbsorptionAx[:,0], meanAllAbsorptionAx[:,1]
plt.plot(meanAllAbsorptionAxData,meanAllAbsorptionAxPos)
plt.title('All Absorption')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Absorption')
plt.savefig('./pics/All/AllAbsorptionAx.png', dpi=300)
plt.show()

# meanAllElastic = dfAllElastic['mean'].values.reshape((500,500))
# plt.imshow(meanAllElastic, interpolation='spline16', cmap='turbo')
# plt.title('All Elastic Scattering')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.colorbar()
# #plt.clim(6e-6,8e-6)
# plt.savefig('./pics/All/AllElastic.png', dpi=300)
# plt.show()

meanAllReactionRate = dfAllReactionRate['mean'].values.reshape((500,500))
plt.imshow(meanAllReactionRate, interpolation='spline16', cmap='turbo')
plt.title('All Reaction Rate')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/All/AllReactionRate.png', dpi=300)
plt.show()

meanAllReactionRateAx = np.column_stack((np.arange(500), meanAllReactionRate[250]))
np.savetxt('./dataFiles/meanAllReactionRateAx.csv', meanAllReactionRateAx, delimiter=',', fmt='%s')
meanAllReactionRateAxData, meanAllReactionRateAxPos = meanAllReactionRateAx[:,0], meanAllReactionRateAx[:,1]
plt.plot(meanAllReactionRateAxData,meanAllReactionRateAxPos)
plt.title('All Reaction Rate')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Reaction Rate')
plt.savefig('./pics/All/AllReactionRateAx.png', dpi=300)
plt.show()

In [ ]:
with openmc.StatePoint('statepoint.{}.h5'.format(settings.batches)) as sp:
    outputThermalTally = sp.get_tally(id=2)
    dfThermal = outputThermalTally.get_pandas_dataframe()
    dfThermalFlux = dfThermal[dfThermal['score'] == 'flux']
    dfThermalFission = dfThermal[dfThermal['score'] == 'nu-fission']
    dfThermalAbsorption = dfThermal[dfThermal['score'] == 'absorption']
    # dfThermalElastic = dfThermal[dfThermal['score'] == '(n,elastic)']
    dfThermalReactionRate = dfThermal[dfThermal['score'] == 'total']

# print(outputThermalTally)
# print(dfThermal)
# print(dfThermalFlux)
# print(dfThermalFission)
# print(dfThermalFlux['mean'])

meanThermalFlux = dfThermalFlux['mean'].values.reshape((500,500))
plt.imshow(meanThermalFlux, interpolation='spline16', cmap='turbo')
plt.title('Thermal Flux')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.savefig('./pics/Thermal/ThermalFlux.png', dpi=300)
#plt.clim(0.0008,0.0012)
plt.show()

meanThermalFluxAx = np.column_stack((np.arange(500), meanThermalFlux[250]))
np.savetxt('./dataFiles/meanThermalFluxAx.csv', meanThermalFluxAx, delimiter=',', fmt='%s')
meanThermalFluxAxData, meanThermalFluxAxPos = meanThermalFluxAx[:,0], meanThermalFluxAx[:,1]
plt.plot(meanThermalFluxAxData,meanThermalFluxAxPos)
plt.title('Thermal Flux')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Flux')
plt.savefig('./pics/Thermal/ThermalFluxAx.png', dpi=300)
plt.show()

meanThermalFission = dfThermalFission['mean'].values.reshape((500,500))
plt.imshow(meanThermalFission, interpolation='spline16', cmap='turbo')
plt.title('Thermal nu-fission')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Thermal/ThermalFission.png', dpi=300)
plt.show()

meanThermalFissionAx = np.column_stack((np.arange(500), meanThermalFission[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanThermalFissionAx, delimiter=',', fmt='%s')
meanThermalFissionAxData, meanThermalFissionAxPos = meanThermalFissionAx[:,0], meanThermalFissionAx[:,1]
plt.plot(meanThermalFissionAxData,meanThermalFissionAxPos)
plt.title('All nu-fission')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('nu-fission')
plt.savefig('./pics/Thermal/ThermalFissionAx.png', dpi=300)
plt.show()

meanThermalAbsorption = dfThermalAbsorption['mean'].values.reshape((500,500))
plt.imshow(meanThermalAbsorption, interpolation='spline16', cmap='turbo')
plt.title('Thermal Absorption')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Thermal/ThermalAbsorption.png', dpi=300)
plt.show()

meanThermalAbsorptionAx = np.column_stack((np.arange(500), meanThermalAbsorption[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanThermalAbsorptionAx, delimiter=',', fmt='%s')
meanThermalAbsorptionAxData, meanThermalAbsorptionAxPos = meanThermalAbsorptionAx[:,0], meanThermalAbsorptionAx[:,1]
plt.plot(meanThermalAbsorptionAxData,meanThermalAbsorptionAxPos)
plt.title('Thermal Absorption')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Absorption')
plt.savefig('./pics/Thermal/ThermalAbsorptionAx.png', dpi=300)
plt.show()

# meanThermalElastic = dfThermalElastic['mean'].values.reshape((500,500))
# plt.imshow(meanThermalElastic, interpolation='spline16', cmap='turbo')
# plt.title('Thermal Elastic Scattering')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.colorbar()
# #plt.clim(6e-6,8e-6)
# plt.savefig('./pics/Thermal/ThermalElastic.png', dpi=300)
# plt.show()

meanThermalReactionRate = dfThermalReactionRate['mean'].values.reshape((500,500))
plt.imshow(meanThermalReactionRate, interpolation='spline16', cmap='turbo')
plt.title('Thermal Reaction Rate')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Thermal/ThermalReactionRate.png', dpi=300)
plt.show()

meanThermalReactionRateAx = np.column_stack((np.arange(500), meanThermalReactionRate[250]))
np.savetxt('./dataFiles/meanThermalReactionRateAx.csv', meanThermalReactionRateAx, delimiter=',', fmt='%s')
meanThermalReactionRateAxData, meanThermalReactionRateAxPos = meanThermalReactionRateAx[:,0], meanThermalReactionRateAx[:,1]
plt.plot(meanAllReactionRateAxData,meanAllReactionRateAxPos)
plt.title('Thermal Reaction Rate')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Reaction Rate')
plt.savefig('./pics/Thermal/ThermalReactionRateAx.png', dpi=300)
plt.show()

In [ ]:
with openmc.StatePoint('statepoint.{}.h5'.format(settings.batches)) as sp:
    outputFastTally = sp.get_tally(id=3)
    dfFast = outputFastTally.get_pandas_dataframe()
    dfFastFlux = dfFast[dfFast['score'] == 'flux']
    dfFastFission = dfFast[dfFast['score'] == 'nu-fission']
    dfFastAbsorption = dfFast[dfFast['score'] == 'absorption']
    # dfFastElastic = dfFast[dfFast['score'] == '(n,elastic)']
    dfFastReactionRate = dfFast[dfFast['score'] == 'total']

# print(outputFastTally)
# print(dfFast)
# print(dfFastFlux)
# print(dfFastFission)
# print(dfFastFlux['mean'])

meanFastFlux = dfFastFlux['mean'].values.reshape((500,500))
plt.imshow(meanFastFlux, interpolation='spline16', cmap='turbo')
plt.title('Fast Flux')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.savefig('./pics/Fast/FastFlux.png', dpi=300)
#plt.clim(0.0008,0.0012)
plt.show()

meanFastFluxAx = np.column_stack((np.arange(500), meanFastFlux[250]))
np.savetxt('./dataFiles/meanFastFluxAx.csv', meanFastFluxAx, delimiter=',', fmt='%s')
meanFastFluxAxData, meanFastFluxAxPos = meanFastFluxAx[:,0], meanFastFluxAx[:,1]
plt.plot(meanFastFluxAxData,meanFastFluxAxPos)
plt.title('Fast Flux')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Flux')
plt.savefig('./pics/Fast/FastFluxAx.png', dpi=300)
plt.show()

meanFastFission = dfFastFission['mean'].values.reshape((500,500))
plt.imshow(meanFastFission, interpolation='spline16', cmap='turbo')
plt.title('Fast nu-fission')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Fast/FastFission.png', dpi=300)
plt.show()

meanFastFissionAx = np.column_stack((np.arange(500), meanFastFission[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanFastFissionAx, delimiter=',', fmt='%s')
meanFastFissionAxData, meanFastFissionAxPos = meanFastFissionAx[:,0], meanFastFissionAx[:,1]
plt.plot(meanFastFissionAxData,meanFastFissionAxPos)
plt.title('All nu-fission')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('nu-fission')
plt.savefig('./pics/Fast/FastFissionAx.png', dpi=300)
plt.show()

meanFastAbsorption = dfFastAbsorption['mean'].values.reshape((500,500))
plt.imshow(meanFastAbsorption, interpolation='spline16', cmap='turbo')
plt.title('Fast Absorption')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Fast/FastAbsorption.png', dpi=300)
plt.show()

meanFastAbsorptionAx = np.column_stack((np.arange(500), meanFastAbsorption[250]))
np.savetxt('./dataFiles/meanAllAbsorptionAx.csv', meanFastAbsorptionAx, delimiter=',', fmt='%s')
meanFastAbsorptionAxData, meanFastAbsorptionAxPos = meanFastAbsorptionAx[:,0], meanFastAbsorptionAx[:,1]
plt.plot(meanFastAbsorptionAxData,meanFastAbsorptionAxPos)
plt.title('Fast Absorption')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Absorption')
plt.savefig('./pics/Fast/FastAbsorptionAx.png', dpi=300)
plt.show()

# meanFastElastic = dfFastElastic['mean'].values.reshape((500,500))
# plt.imshow(meanFastElastic, interpolation='spline16', cmap='turbo')
# plt.title('Fast Elastic Scattering')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.colorbar()
# #plt.clim(6e-6,8e-6)
# plt.savefig('./pics/Fast/FastElastic.png', dpi=300)
# plt.show()

meanFastReactionRate = dfFastReactionRate['mean'].values.reshape((500,500))
plt.imshow(meanFastReactionRate, interpolation='spline16', cmap='turbo')
plt.title('Fast Reaction Rate')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
#plt.clim(6e-6,8e-6)
plt.savefig('./pics/Fast/FastReactionRate.png', dpi=300)
plt.show()

meanFastReactionRateAx = np.column_stack((np.arange(500), meanFastReactionRate[250]))
np.savetxt('./dataFiles/meanFastReactionRateAx.csv', meanFastReactionRateAx, delimiter=',', fmt='%s')
meanFastReactionRateAxData, meanFastReactionRateAxPos = meanFastReactionRateAx[:,0], meanFastReactionRateAx[:,1]
plt.plot(meanAllReactionRateAxData,meanAllReactionRateAxPos)
plt.title('Fast Reaction Rate')
plt.xlabel('x')
#plt.ylim(bottom=0.0008)
plt.ylabel('Reaction Rate')
plt.savefig('./pics/Fast/FastReactionRateAx.png', dpi=300)
plt.show()